# Продажи австралийского вина
Известны ежемесячные продажи австралийского вина в тысячах литров с января 1980 по июль 1995, необходимо построить прогноз на следующие два года.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [ ]:
data = pd.read_csv('monthly-australian-wine-sales-th.csv', parse_dates=[0], delimiter=',')
data.head()


In [ ]:
data.columns = ['date', 'wine']
data.head()

In [ ]:
sns.lineplot(data['date'],data['wine'])

Попробуем поделить на число дней в месяце: 

In [ ]:
values = data['wine'].values.copy()
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
for i in range(len(values)):
    values[i]/=days[i%12]
sns.lineplot(data['date'],values)

Ряд не стал более регулярным, так что вернёмся к исходным данным.

STL-декомпозиция ряда:

In [ ]:
data = data.set_index(pd.DatetimeIndex(data['date']))

In [ ]:
data.head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
stl_data = seasonal_decompose(data['wine'])
_= stl_data.plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
transformed,lam = st.boxcox(data['wine'])
data['bc'] = transformed
print (lam)
stl_data = seasonal_decompose(transformed, freq=12)
_= stl_data.plot()

# ARIMA
Стационарность:

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss(data['wine'])

Cделаем сезонное дифференцирование: 

In [ ]:
diff_ts = data['wine'].diff(12)[12:]
plt.plot(diff_ts)
kpss(diff_ts)

Ряд всё ещё нестационарен. Проведём ещё одно дифференцирование: 

In [ ]:
diff_diff_ts = diff_ts.diff(1)[1:]
plt.plot(diff_diff_ts)
kpss(diff_diff_ts)

Для полученного ряда гипотеза стационарности не отвергается

Посмотрим на ACF и PACF полученного продифференцированного ряда:

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
_ = plot_acf(diff_diff_ts, lags=15)


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
_ = plot_pacf(diff_diff_ts, lags=15)


На ACF значимы лаги 1 и 12, на PACF — 1-3. Будем искать модель, оптимальную по AICc, в окрестности ARIMA(3,1,1)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
for i in range(-1,2):
        for k in range(-1, 2):
            order = (3+i,1,1+k)
            try:
                # ваш код                
            except:
                pass 

In [ ]:
arima = ARIMA(data['wine'], order=(2,1,1), freq='MS').fit()
arima.summary2()


In [ ]:
plt.plot(arima.resid)
_=plot_pacf(arima.resid, lags=15)
_=plot_acf(arima.resid, lags=15)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
for i in range(-1,2):
    for j in range(-1, 2):
            order = (3+i,1,1+j)
            sorder = (1,1,1,12)
            # ваш код 

Наилучшая по AIC модель — ARIMA(3,1,1)(1,1,1, 12). Посмотрим на её остатки: 

In [ ]:
arima = SARIMAX(data['wine'], order=(3,1,1), seasonal_order=(1,1,1,12)).fit()


plt.plot(arima.resid)
_=plot_pacf(arima.resid, lags=15)
_=plot_acf(arima.resid, lags=15)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
plt.scatter(range(16), acorr_ljungbox(arima.resid, lags=16)[1])


Q-Q plot: 

In [ ]:
_ = st.probplot(arima.resid, plot=plt)
print (st.shapiro(arima.resid))
plt.show()
plt.hist(arima.resid)

In [ ]:
arima = SARIMAX(data['wine'][:-12], order=(3,1,1), seasonal_order=(1,1,1,12)).fit()
predicted = arima.predict(0, data.shape[0])

In [ ]:
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)


In [ ]:
arima = SARIMAX(data['wine'][:-12], order=(3,1,1), seasonal_order=(1,1,1,12)).fit()
predicted = arima.predict( data.shape[0]-12, data.shape[0], dynamic=True)

In [ ]:
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)


# Прогноз ETS

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
ets = ExponentialSmoothing(data['wine'][12:]).fit(optimized=True)

predicted = ets.predict(0, data.shape[0])
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)


In [ ]:
ets = ExponentialSmoothing(data['wine'][12:],  trend='add', seasonal_periods=12).fit()

predicted = ets.predict(0, data.shape[0])
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)
ets.aic

In [ ]:
ets = ExponentialSmoothing(data['wine'][12:], seasonal='mul', trend='add', seasonal_periods=12).fit()

predicted = ets.predict(0, data.shape[0])
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)
ets.aic

In [ ]:
_ = st.probplot(ets.resid, plot=plt)
print (st.shapiro(ets.resid))
plt.show()
plt.hist(ets.resid)

In [ ]:
ets = ExponentialSmoothing(data['wine'][12:], seasonal='mul', trend='add', seasonal_periods=12).fit(use_boxcox=True)

predicted = ets.predict(0, data.shape[0])
plt.plot(data['wine'][:-12], alpha=0.5, ls='--')
plt.plot(data['wine'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)
ets.aic

In [ ]:
_ = st.probplot(ets.resid, plot=plt)
print (st.shapiro(ets.resid))
plt.show()
plt.hist(ets.resid)

In [ ]:
ets = ExponentialSmoothing(data['bc'][12:], seasonal='mul', trend='add', seasonal_periods=12).fit()

predicted = ets.predict(0, data.shape[0])
plt.plot(data['bc'][:-12], alpha=0.5, ls='--')
plt.plot(data['bc'][-12:], alpha=0.5, ls='--')
plt.plot(predicted)
ets.aic

In [ ]:
_ = st.probplot(ets.resid, plot=plt)
print (st.shapiro(ets.resid))
plt.show()
plt.hist(ets.resid)